## 1. Mount Drive

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Import libraries

In [2]:
!pip3 install rdflib

from rdflib import Graph, Literal, BNode, Namespace, RDF, URIRef
from rdflib.namespace import DC, FOAF

import json
import pprint
import re

## 3. Load existing Ontology

In [0]:
#tutorial: https://rdflib.readthedocs.io/en/stable/index.html
g = Graph()
# Load Ontology Framework from "cyclistDB.owl" so it can be enriched with new data
with open("/content/drive/My Drive/Semantic Webtechnology #JodyBernal 🦄/cyclistDB.owl", "r") as f:
    g.parse(f, format="xml")
    '''
    for stmt in g:
        pprint.pprint(stmt)
    '''

## 4. Enrich existing Ontology 
Add Teams and Bike_Sponsors with corresponding object and data properties to Ontology

In [4]:
with open("/content/drive/My Drive/Semantic Webtechnology #JodyBernal 🦄/code/enriched_teams.json","r") as f:
    team_data = json.load(f)
    n = Namespace('http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#')
    s = Namespace('http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#')

    # TODO: change keys
    # TODO: add nationality for Teams dbp:registered
    for team in team_data.keys():
        #print(team_data[team])
        # Create URI for Team, with team name separated by underscores instead of spaces
        team_name = team.rstrip()
        team_name = team_name.replace(" ","_")
        teamURI = URIRef(n+team_name)
        g.add([teamURI, RDF.type, URIRef('http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#Team')])

        # Add Bike_Sponsor to ontology and add "Sponsored_By" relation between Team and Bike_Sponsor
        Bike_Sponsor = URIRef("http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#Bike_Sponsor")
        Sponsors =  URIRef("http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#Sponsors")
        Sponsored_By = URIRef("http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#Sponsored_By")
        if team_data[team]["dbp:bicycles"] != None:
            Sponsor = team_data[team]["dbp:bicycles"].rstrip().replace(" ","_")
            SponsorURI = URIRef(s+Sponsor)
            g.add([SponsorURI, RDF.type, Bike_Sponsor])
            g.add([teamURI, Sponsored_By, SponsorURI])
            g.add([SponsorURI, Sponsors, teamURI])

        # Add uciCode property of Team
        uciCode = URIRef("http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#uciCode")
        g.add([teamURI, uciCode, Literal(team_data[team]["dbo:uciCode"])])

        # Add Status property of Team
        Status = URIRef("http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#Status")
        g.add([teamURI, Status, Literal(team_data[team]["dbo:status"])])

        Nationality = URIRef("http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#Nationality")
        if team_data[team]["dbp:registered"] != None:
            g.add([teamURI, Nationality, Literal(team_data[team]["dbp:registered"])])


http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#<a_href='http://dbpedia.org/resource/Specialized_Bicycle_Components'>Specialized_Bicycle_Components</a> does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#<a_href='http://dbpedia.org/resource/Colnago'>Colnago</a> does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#<a_href='http://dbpedia.org/resource/Genesis_(bikes)'>Genesis_(bikes)</a> does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#<a_href='http://dbpedia.org/resource/Focus_Bikes'>Focus_Bikes</a> does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#<a_href='http://dbpedia.org/resource/Canyon_Bicycles'>Canyon_Bicycles</a> does not look like a valid URI, t

Add Riders with corresponding object and data properties to Ontology

In [6]:
with open("/content/drive/My Drive/Semantic Webtechnology #JodyBernal 🦄/code/enriched_riders.json","r") as f:
    rider_data = json.load(f) 
    n = Namespace('http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#')
    t = Namespace('http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#')

    for rider in rider_data.keys():
        #print(team_data[rider])
        # Create URI for Rider, with full Rider name separated by underscores
        rider_name = rider.rstrip()
        rider_name = rider_name.replace(" ", "_")
        riderURI = URIRef(n+rider_name)
        g.add([riderURI, RDF.type, URIRef("http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#Rider")])
        
        # Add FullName property of Rider
        FullName = URIRef("http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#FullName")
        g.add([riderURI, FullName, Literal(rider_name) ])
        # Add GivenName property of Rider
        GivenName = URIRef("http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#GivenName") 
        g.add([riderURI, GivenName, Literal(rider_data[rider]["foaf:givenName"])])
        # Add Surname property of Rider
        Surname = URIRef("http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#Surname")
        g.add([riderURI, Surname, Literal(rider_data[rider]["foaf:surname"])])

        # Add UCI_rank property of Rider
        UCI_rank = URIRef("http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#UCI_rank")
        if rider_data[rider]["dbp:uciRank"] != None:
            g.add([riderURI, UCI_rank, Literal(rider_data[rider]["dbp:uciRank"])])
        # Add Height property of Rider
        Height = URIRef("http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#Height")
        if rider_data[rider]["dbo:height"] != None:
            g.add([riderURI, Height, Literal(float(rider_data[rider]["dbo:height"]))])
        # Add Weight property of Rider
        Weight = URIRef("http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#Weight")
        if rider_data[rider]["dbo:weight"] != None:
            g.add([riderURI, Weight, Literal(rider_data[rider]["dbo:weight"])])

        # Add Nationality property of Rider
        Nationality = URIRef("http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#Nationality")
        g.add([riderURI, Nationality, Literal(rider_data[rider]["dbo:nationality"])])
        # Add BirthPlace property of Rider
        BirthPlace = URIRef("http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#BirthPlace")
        if rider_data[rider]["dbo:birthPlace"] != None:
            g.add([riderURI, BirthPlace, Literal(rider_data[rider]["dbo:birthPlace"])])
            
        # Add BirthDate property of Rider
        BirthDate = URIRef("http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#BirthDate")
        if rider_data[rider]["dbo:birthDate"] != None:
            g.add([riderURI, BirthDate, Literal(rider_data[rider]["dbo:birthDate"])])

        # Add RiderType property of Rider
        RiderType = URIRef("http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#RiderType")
        if rider_data[rider]["dbp:ridertype"] != None:
            g.add([riderURI, RiderType, Literal(rider_data[rider]["dbp:ridertype"])])

        # Add ProTeam of Rider
        Rides_For = URIRef("http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#Rides_For")
        Has_Rider = URIRef("http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#Has_Rider")
        if rider_data[rider]["dbp:proteam"] != None:
            # In case the proteam is a URL, filter out the team name and store URL as property
            if rider_data[rider]["dbp:proteam"][0] == "<":
                name_match = re.match(r"^.*\>(.*)\<.*$",rider_data[rider]["dbp:proteam"])
                team_name = name_match.group(1)
                team_name = team_name.rstrip().replace(" ", "_")
                link_match = re.match(r"^.*\='(.*)'\>.*$",rider_data[rider]["dbp:proteam"])
                DBpedia_link = link_match.group(1)
                URL =  URIRef("http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#URL")
                #g.add([t+team_name, RDF.type, URIRef('http://www.semanticweb.org/bertelt/ontologies/2019/8/cyclistDB#Team')])
                g.add([URIRef(t+team_name), URL, Literal(str(DBpedia_link))])
            else:
                team_name = rider_data[rider]["dbp:proteam"].rstrip()
                team_name = team_name.replace(" ","_")
            ProTeamURI = URIRef(t+team_name)
            g.add([riderURI, Rides_For, ProTeamURI])
            g.add([ProTeamURI, Has_Rider, riderURI])

    #for stmt in g:
    #    pprint.pprint(stmt)
        
    #print("graph has %s statements." % len(g))

Jesús Ezquerra
Awet Gebremedhin
Jonas Vingegaard
Yevgeniy Gidich
Michael Valgren
German Nicolás Tivani
Ilnur Zakarin
Jan Stöhr
Jhonatan Narváez
Patrick Haller
Omar Fraile
Edvald Boasson Hagen
Andrea Guardini
Andrey Zeits
Nathan Van Hooydonck


## 5. Store enrichted Ontology
Store Ontology enrichted with ProCyclingStats data in "ProcyclingStatsDB.owl"

In [0]:
with open("/content/drive/My Drive/Semantic Webtechnology #JodyBernal 🦄/ProcyclingStatsDB.owl","wb") as f:
    g.serialize(destination=f, format="xml")